<a href="https://colab.research.google.com/github/tomek-l/fire/blob/master/3_5_1EntireDryCreekGradCAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/gdrive', force_remount=False)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
!ls  '/gdrive/My Drive/FireVideo/dry_creek/dry_creek_30fps' | wc

ls: cannot access '/gdrive/My Drive/FireVideo/dry_creek/dry_creek_30fps': No such file or directory
      0       0       0


In [0]:
from keras.models import load_model
model = load_model('/gdrive/My Drive/FireVideo/models/VGG_monolth_30epochs.h5')



Using TensorFlow backend.
W0710 21:18:21.360661 139717572421504 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0710 21:18:21.458821 139717572421504 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0710 21:18:21.586257 139717572421504 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0710 21:18:22.074877 139717572421504 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0710 21:18:22.076260 13

In [0]:
from keras.preprocessing import image
import numpy as np

def get_processed_image(img_path):
  img = image.load_img(img_path, target_size=(150, 150))
  img_tensor = image.img_to_array(img)
  img_tensor = np.expand_dims(img_tensor, axis=0)
  img_tensor /= 255. #(the network is trained on 0.0-1.0 inputs)
  return img_tensor

In [0]:
#Build graph for CAM-grad process
import keras.backend as K

binary_fire_output = model.output
last_conv_layer = model.get_layer('block5_conv3')
grads = K.gradients(model.output, last_conv_layer.output)[0]
print(grads)
pooled_grads = K.mean(grads, axis=(0, 1, 2))
print(pooled_grads)

Tensor("gradients/block5_pool/MaxPool_grad/MaxPoolGrad:0", shape=(?, 9, 9, 512), dtype=float32)
Tensor("Mean:0", shape=(512,), dtype=float32)


In [0]:
import os
import cv2
import matplotlib.pyplot as plt

dir = '/gdrive/My Drive/FireVideo/dry_creek/dry_creek_30fps/'
cnt = 0
for idx, name in enumerate(os.listdir(dir)):
 
#   if idx == 100: break # do just 5 examples
  img_path = dir + name
  img_tensor = get_processed_image(img_path)

  iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])
  pooled_grads_value, conv_layer_output_value = iterate([img_tensor])
  for i in range(512):
      conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

  heatmap = np.mean(conv_layer_output_value, axis=-1)

  heatmap_amplitude = np.max(heatmap)-np.min(heatmap)
  min_heatmap = np.min(heatmap)

  heatmap = (heatmap+np.abs(min_heatmap))/heatmap_amplitude

  img = cv2.imread(img_path)
  grayimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  grayimg = cv2.cvtColor(grayimg, cv2.COLOR_GRAY2BGR)
  
  rgbimg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
  heatmap = np.uint8(255 * heatmap)

  heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
  superimposed_img = np.uint8(0.5* heatmap + 0.5* grayimg)
  superimposed_img = cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB)
  
  
  cv2.imwrite('/gdrive/My Drive/FireVideo/dry_creek/gradCAM_30fps/'+name.split('.')[0]+'_heatmap.jpg',
              cv2.cvtColor(superimposed_img, cv2.COLOR_RGB2BGR))
  
  print('{}/7927'.format(cnt))
  cnt +=1
  #cv2.imwrite('/gdrive/My Drive/FireVideo/dry_creek/gradCAM_30fps/'+name, img)



0/7927
1/7927
2/7927
3/7927
4/7927
5/7927
6/7927
7/7927
8/7927
9/7927
10/7927
11/7927
12/7927
13/7927
14/7927
15/7927
16/7927
17/7927
18/7927
19/7927
20/7927
21/7927
22/7927
23/7927
24/7927
25/7927
26/7927
27/7927
28/7927
29/7927
30/7927
31/7927
32/7927
33/7927
34/7927
35/7927
36/7927
37/7927
38/7927
39/7927
40/7927
41/7927
42/7927
43/7927
44/7927
45/7927
46/7927
47/7927
48/7927
49/7927
50/7927
51/7927
52/7927
53/7927
54/7927
55/7927
56/7927
57/7927
58/7927
59/7927
60/7927
61/7927
62/7927
63/7927
64/7927
65/7927
66/7927
67/7927
68/7927
69/7927
70/7927
71/7927
72/7927
73/7927
74/7927
75/7927
76/7927
77/7927
78/7927
79/7927
80/7927
81/7927
82/7927
83/7927
84/7927
85/7927
86/7927
87/7927
88/7927
89/7927
90/7927
91/7927
92/7927
93/7927
94/7927
95/7927
96/7927
97/7927
98/7927
99/7927
100/7927
101/7927
102/7927
103/7927
104/7927
105/7927
106/7927
107/7927
108/7927
109/7927
110/7927
111/7927
112/7927
113/7927
114/7927
115/7927
116/7927
117/7927
118/7927
119/7927
120/7927
121/7927
122/7927
123

In [0]:
!ffmpeg -framerate 30 -pattern_type glob -i '/gdrive/My Drive/FireVideo/dry_creek/gradCAM_30fps/*.jpg' -c:v libx264 -r 30 -pix_fmt yuv420p '/gdrive/My Drive/FireVideo/dry_creek/out_full.mp4'

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena

In [0]:
!ls '/gdrive/My Drive/FireVideo/dry_creek/gradCAM_30fps/'

out00001_heatmap.jpg  out00035_heatmap.jpg  out00069_heatmap.jpg
out00002_heatmap.jpg  out00036_heatmap.jpg  out00070_heatmap.jpg
out00003_heatmap.jpg  out00037_heatmap.jpg  out00071_heatmap.jpg
out00004_heatmap.jpg  out00038_heatmap.jpg  out00072_heatmap.jpg
out00005_heatmap.jpg  out00039_heatmap.jpg  out00073_heatmap.jpg
out00006_heatmap.jpg  out00040_heatmap.jpg  out00074_heatmap.jpg
out00007_heatmap.jpg  out00041_heatmap.jpg  out00075_heatmap.jpg
out00008_heatmap.jpg  out00042_heatmap.jpg  out00076_heatmap.jpg
out00009_heatmap.jpg  out00043_heatmap.jpg  out00077_heatmap.jpg
out00010_heatmap.jpg  out00044_heatmap.jpg  out00078_heatmap.jpg
out00011_heatmap.jpg  out00045_heatmap.jpg  out00079_heatmap.jpg
out00012_heatmap.jpg  out00046_heatmap.jpg  out00080_heatmap.jpg
out00013_heatmap.jpg  out00047_heatmap.jpg  out00081_heatmap.jpg
out00014_heatmap.jpg  out00048_heatmap.jpg  out00082_heatmap.jpg
out00015_heatmap.jpg  out00049_heatmap.jpg  out00083_heatmap.jpg
out00016_heatmap.jpg  out